In [6]:
import pandas as pd
import numpy as np

In [60]:
# Loading all data bases 

branchements = pd.read_csv('../data_SUEZ/branchements.csv')
canalisations = pd.read_csv('../data_SUEZ/canalisations.csv')
evenements_train = pd.read_csv('../data_SUEZ/evenements_train.csv')
meteo_AlpesMaritimes = pd.read_csv('../data_SUEZ/meteo_AlpesMaritimes_pre_traite.csv')
nb_ev_train = pd.read_csv('../data_SUEZ/nb_ev_train.csv')
permut_train = pd.read_csv('../data_SUEZ/permut_train.csv')
stations_AlpesMaritimes = pd.read_csv('../data_SUEZ/stations_AlpesMaritimes.csv')

In [61]:
# Renaming columns

evenements_train.columns= ['GID_fuite', 'CODINSEE_fuite', 'date_fuite', 'GID_pose', 'COMPO_P', 'geometry_fuite']
branchements.columns= ['GID_pose', 'CODINSEE_pose', 'date_pose', 'DIAMETRE', 'DIAMEXT', 'DIAMINT', 'ETAGE',
       'GEORIENT', 'LONGCALC', 'MATERIAU', 'geometry_pose']
canalisations.columns= ['GID_pose', 'CODINSEE_pose', 'date_pose', 'DIAMETRE', 'DIAMEXT', 'DIAMINT', 'ETAGE',
       'GEORIENT', 'LONGCALC', 'MATERIAU', 'geometry_pose']

In [64]:
evenements_train['ajout_branch'] = '_b'
evenements_train['ajout_can'] = '_c'
evenements_train.ix[(evenements_train.COMPO_P == 'E_BRANCH'),'GID_pose'] = evenements_train.ix[(evenements_train.COMPO_P == 'E_BRANCH'),'GID_pose'].map(str) + evenements_train.ix[(evenements_train.COMPO_P == 'E_BRANCH'),'ajout_branch']
evenements_train.ix[(evenements_train.COMPO_P == 'E_TRONCO'),'GID_pose'] = evenements_train.ix[(evenements_train.COMPO_P == 'E_TRONCO'),'GID_pose'].map(str) + evenements_train.ix[(evenements_train.COMPO_P == 'E_TRONCO'),'ajout_can']
evenements_train.drop('ajout_branch', axis=1, inplace=True)
evenements_train.drop('ajout_can', axis=1, inplace=True)

branchements['ajout_branch'] = '_b'
branchements.GID_pose = branchements.GID_pose.map(str) + branchements.ajout_branch
branchements.drop('ajout_branch', axis=1, inplace=True)

canalisations['ajout_can'] = '_c'
canalisations.GID_pose = canalisations.GID_pose.map(str) + canalisations.ajout_can
canalisations.drop('ajout_can', axis=1, inplace=True)

In [65]:
# Creating COMPO_P column in canalisation and branchements 

canalisations['COMPO_P'] = 'E_TRONCO'
branchements['COMPO_P'] = 'E_BRANCH'
elements = pd.concat([canalisations, branchements])

In [66]:
# Merge elements on evenements (creation BDD complète)

elements_train = pd.merge(elements, evenements_train, on = 'GID_pose', how = 'left')

In [64]:
# Create BDD modèle 1ère fuite

elements_train['fuite'] = 
elements_train['GID_pose' elements_train['GID_fuite'].isnull().sum()

79076

In [27]:
# Put info from branchements in evenements_br 

merge_br = pd.merge(evenements_train_br, branchements, on = 'GID_pose')
sum(merge_br.CODINSEE_fuite != merge_br.CODINSEE_pose)

5

In [28]:
# Put info from canalisation in evenements_canal 

merge_canal = pd.merge(evenements_train_canal, canalisations, on = 'GID_pose')
sum(merge_canal.CODINSEE_fuite != merge_canal.CODINSEE_pose)

3

In [63]:
elements_train['CODINSEE_fuite'].isin([06003, 06004, 06016, 06030, 06070, 06095, 06108, 06112, 06116, 06138, 06152]
])

SyntaxError: invalid token (<ipython-input-63-2407e25f3a1b>, line 1)